<a href="https://colab.research.google.com/github/danielsyahputra13/ml_capstone/blob/master/notebooks/Copy_of_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/Shareddrives/ML-Capstone/dataset/train.csv")
test = pd.read_csv("/content/drive/Shareddrives/ML-Capstone/dataset/test.csv")

In [ ]:
data.head()

,drugName,condition,review,rating,date,usefulCount,clean_review
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,effect combin bystol mg fish oil
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,son halfway fourth week intuniv concern began ...
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,oral contracept pill cycl happi light period m...
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,time form birth control m glad went patch mont...
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,suboxon complet turn life feel healthier m exc...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160383 entries, 0 to 160382
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   drugName      160383 non-null  object 
 1   condition     160383 non-null  object 
 2   review        160383 non-null  object 
 3   rating        160383 non-null  float64
 4   date          160383 non-null  object 
 5   usefulCount   160383 non-null  int64  
 6   clean_review  160383 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 8.6+ MB


In [ ]:
# data['num_of_words'] = data['clean_review'].apply(lambda x: len(nltk.word_tokenize(x)))

In [ ]:
# data['num_of_words'].describe()

In [ ]:
data  = data[data['condition'] != 'me']
data = data[data['condition'] != 'mist (']

In [ ]:
data = data[~data['condition'].str.contains('</span>')]

In [ ]:
test  = test[test['condition'] != 'me']
test = test[test['condition'] != 'mist (']
test = test[~test['condition'].str.contains('</span>')]

In [ ]:
test = test[test['condition'].isin(data['condition'])]

# Model


In [ ]:
vocab_size = 5000
embedding_dim = 64
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

Split data train vs val

In [ ]:
# from sklearn.model_selection import train_test_split

# train, val = train_test_split(data, test_size=0.2)
train = data.copy()
test = test.copy()

In [ ]:
print("Train size:", len(train))

Train size: 159481


In [ ]:
print("Testing size:", len(test))

Testing size: 53156


In [ ]:
train_reviews = train['clean_review'].tolist()
test_reviews = test['clean_review'].tolist()

## Transformasi data train

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'day': 3,
 'effect': 6,
 'm': 10,
 'month': 4,
 'start': 9,
 't': 2,
 'week': 8,
 'work': 7,
 'year': 5}

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_reviews)
print(train_sequences[10])

[23, 8, 9, 17, 7, 125, 17, 362, 17, 60, 79, 49, 1496, 110, 8, 89, 85, 79, 889, 365, 525, 1290, 73, 68, 26, 179, 26, 165, 6, 63, 22, 6, 974, 1708, 124, 966, 645, 124, 146, 578, 1809, 14, 3758, 10, 122, 960, 44, 1817, 387, 891, 212, 46, 70]


In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

6
100
52
100
53
100


## Transformasi data validation

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_reviews)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(test_sequences))
print(test_padded.shape)

53156
(53156, 100)


## Extract Label

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [ ]:
label_train = train['condition'].values
label_train

array(['Left Ventricular Dysfunction', 'ADHD', 'Birth Control', ...,
       'Rheumatoid Arthritis', 'Underactive Thyroid',
       'Constipation, Chronic'], dtype=object)

In [ ]:
enc_label_train = encoder.fit_transform(label_train)
enc_label_train

array([393,   0,  92, ..., 628, 738, 160])

In [ ]:
np.save('/content/drive/Shareddrives/ML-Capstone/classes.npy', encoder.classes_)

In [ ]:
new_encoder = LabelEncoder()
new_encoder.classes_ = np.load('/content/drive/Shareddrives/ML-Capstone/classes.npy', allow_pickle=True)

In [ ]:
encoder.inverse_transform([123])

array(['Cance'], dtype=object)

In [ ]:
new_encoder.inverse_transform([123])

array(['Cance'], dtype=object)

In [ ]:
train['condition'].nunique()

808

In [ ]:
label_test = test['condition'].values
label_test

array(['Depression', "Crohn's Disease, Maintenance",
       'Urinary Tract Infection', ..., 'Birth Control', 'Pain',
       'Sciatica'], dtype=object)

In [ ]:
enc_label_test = encoder.transform(label_test)
enc_label_test

array([189, 169, 744, ...,  92, 521, 642])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(808, activation='softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 64)          320000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 808)               26664     
                                                                 
Total params: 416,840
Trainable params: 416,840
Non-trainable params: 0
________________________________________________

In [ ]:
print(len(set(enc_label_train)))
print(len(set(enc_label_test)))


808
637


In [ ]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if epoch >= 15:
          if logs.get('val_acc') < 0.7: 
            print("\nReached %2.2f%% accuracy, stopping training!!" %(logs.get('val_acc')*100))   
            self.model.stop_training = True
          else:
            pass

callbacks = myCallback()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 30
history = model.fit(train_padded,
                    enc_label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, enc_label_test),
                    verbose=1, 
                    batch_size=512)

Epoch 1/30
312/312 [==============================] - 316s 890ms/step - loss: 4.3600 - accuracy: 0.1915 - val_loss: 3.5728 - val_accuracy: 0.2597
Epoch 2/30
312/312 [==============================] - 276s 884ms/step - loss: 3.2066 - accuracy: 0.3147 - val_loss: 2.8347 - val_accuracy: 0.3866
Epoch 3/30
312/312 [==============================] - 273s 875ms/step - loss: 2.6589 - accuracy: 0.4234 - val_loss: 2.4184 - val_accuracy: 0.4775
Epoch 4/30
312/312 [==============================] - 275s 880ms/step - loss: 2.3029 - accuracy: 0.4945 - val_loss: 2.1564 - val_accuracy: 0.5321
Epoch 5/30
312/312 [==============================] - 277s 888ms/step - loss: 2.0558 - accuracy: 0.5423 - val_loss: 1.9538 - val_accuracy: 0.5705
Epoch 6/30
312/312 [==============================] - 276s 886ms/step - loss: 1.8695 - accuracy: 0.5789 - val_loss: 1.8194 - val_accuracy: 0.5989
Epoch 7/30
312/312 [==============================] - 272s 873ms/step - loss: 1.7291 - accuracy: 0.6049 - val_loss: 1.7069 -

- mist (
- yang ada `</span>`
- `me`
- `min / rosiglitazone)` -> `Rosiglitazon`

In [ ]:
model.save("/content/drive/Shareddrives/ML-Capstone/model/model_daniel_ver3.h5")